In [1]:
import os
import zipfile
import random
import shutil
from os import getcwd
from shutil import copyfile
import tensorflow as tf
from PIL import UnidentifiedImageError
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

In [13]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "cats-and-dogs.zip"

local_zip = "cats-and-dogs.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./')
zip_ref.close()
/home/oluwaseun/.cache/JetBrains/PyCharmCE2021.1/certification/Category2/mymodel.h5

--2021-05-10 19:02:30--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 104.85.0.109, 2a02:26f0:60:895::e59, 2a02:26f0:60:8a3::e59
Connecting to download.microsoft.com (download.microsoft.com)|104.85.0.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘cats-and-dogs.zip’

cats-and-dogs.zip   100%[===================>] 786.68M  2.03MB/s    in 8m 53s  

2021-05-10 19:11:26 (1.48 MB/s) - ‘cats-and-dogs.zip’ saved [824894548/824894548]



In [3]:
print(len(os.listdir('PetImages/Cat/')))
print(len(os.listdir('PetImages/Dog/')))
print(getcwd())

12501
12501
/home/oluwaseun/Documents/Personal_projects/deeplearning.ai_tensorflow_certification


In [ ]:
paths = ["cats-v-dogs/training", "cats-v-dogs/testing"]
for path in paths:
    for i in ["cats", "dogs"]:
        os.makedirs(os.path.join(path, i)) 

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    x = os.listdir(SOURCE)
    files = random.sample(x, len(x))
    num = int(len(files)*SPLIT_SIZE)
    
    train = [files[:num], TRAINING ]
    test = [files[num:], TESTING]
    
    for data in [train,test]:
        for img in data[0]:
            shutil.move(f"{SOURCE}{img}",data[1])

CAT_SOURCE_DIR = "PetImages/Cat/"
TRAINING_CATS_DIR = "cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "PetImages/Dog/"
TRAINING_DOGS_DIR = "cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "cats-v-dogs/testing/dogs/"
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [4]:
print(len(os.listdir('cats-v-dogs/training/cats/')))
print(len(os.listdir('cats-v-dogs/training/dogs/')))
print(len(os.listdir('cats-v-dogs/testing/cats/')))
print(len(os.listdir('cats-v-dogs/testing/dogs/')))

11250
11250
1251
1251


In [47]:
model = tf.keras.models.Sequential([
    Conv2D(16, 3, activation="relu", input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, 3, activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(64, 3, activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 18496)            

In [48]:
TRAINING_DIR = "cats-v-dogs/training"
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True)

# NOTE: YOU MUST USE A BATCH SIZE OF 10 (batch_size=10) FOR THE 
# TRAIN GENERATOR.
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (150,150),
    batch_size =  32,
    class_mode = "binary"
    )

VALIDATION_DIR = "cats-v-dogs/testing"
validation_datagen = ImageDataGenerator(rescale = 1./255)

# NOTE: YOU MUST USE A BACTH SIZE OF 10 (batch_size=10) FOR THE 
# VALIDATION GENERATOR.
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (150,150),
    batch_size =  32,
    class_mode = "binary"
    )



Found 22497 images belonging to 2 classes.
Found 2502 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(train_generator,
                              epochs=2,
                              verbose=1,
                              validation_data=validation_generator)

In [53]:
history.history

{'loss': [0.6935679316520691,
  0.6935516595840454,
  0.6935350894927979,
  0.6935165524482727,
  0.6934945583343506],
 'acc': [0.4936051070690155,
  0.49480414390563965,
  0.49440446496009827,
  0.4972022473812103,
  0.49680256843566895],
 'lr': [1e-08, 1.1220185e-08, 1.2589254e-08, 1.41253755e-08, 1.5848931e-08]}

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')
